In [2]:
from clf_funcs import fit, test, get_cifar10_loaders, get_mnist_loaders, FullyConnectedNet, SimpleConvNet
from dcgan_funcs import fit_dcgan, get_celeba_loader, get_celeba_loader_from_memory_old, Discriminator, Generator, dcgan_weights_init, generate

import numpy as np
import pandas as pd
import seaborn as sns
from torchsummary import summary
from matplotlib import pyplot as plt
from functools import reduce
flat_map = lambda f, xs: reduce(lambda a, b: a + b, map(f, xs), [])

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.models import mobilenet_v2, resnet50

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'CUDA enabled: {use_cuda}')

CUDA enabled: True


In [16]:
dl = get_celeba_loader(96, root='../../datasets/celeba_trunc/')
all_batches = [batch[0] for batch in dl]
all_batches = []
for i, batch in enumerate(dl):
	if i % 50 == 0: print(f'processing batch no. {i}')
	all_batches.append(batch[0])

processing batch no. 0
processing batch no. 50
processing batch no. 100
processing batch no. 150
processing batch no. 200
processing batch no. 250
processing batch no. 300
processing batch no. 350
processing batch no. 400
processing batch no. 450
processing batch no. 500


In [18]:
collected = torch.concat(all_batches, 0)

torch.Size([50000, 3, 64, 64])

In [2]:
def get_celeba_loader_from_memory(batch_size, image_size=64, root='../../datasets/celeba'):
	dl = get_celeba_loader(batch_size, image_size=image_size, root=root)

	# all_batches = [batch[0] for batch in dl]
	collected_batches = []
	for i, batch in enumerate(dl):
		if i % 50 == 0: print(f'processing batch no. {i}')
		collected_batches.append(batch)

	return collected_batches

In [2]:
celeba_dl = get_celeba_loader_from_memory(96, root='../../datasets/celeba_test')

In [7]:
celeba_dl[0][0].shape, celeba_dl[0][1]

(torch.Size([96, 3, 64, 64]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [2]:
batch_size = 128
test_batch_size = 128
epochs = 2
lr = 1e-2
momentum = 0.9
num_classes = 10
log_interval = 300

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f'CUDA enabled: {use_cuda}')

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

CUDA enabled: True


In [ ]:
model = mobilenet_v2()
model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
model.modules

In [ ]:
# model = mobilenet_v2()
# model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
train_dl, test_dl = get_cifar10_loaders(batch_size, test_batch_size)
loss_func = F.cross_entropy
opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

model = model.to(device)

for epoch in range(1, epochs + 1):

	train_history = fit(model, device, train_dl, loss_func, epoch, optimizer=opt, log_interval=log_interval, silent=False)
	_, accuracy = test(model, device, test_dl, loss_func, silent=False)

In [7]:
nc = 3
nz = 100
ngf = 64
ndf = 64

batch_size = 128
epochs = 3
lr = 1e-2
log_interval = 50

In [6]:
netG = Generator(nc, nz, ngf).to(device)
netD = Discriminator(nc, ndf).to(device)
netG.apply(dcgan_weights_init)
netD.apply(dcgan_weights_init)

# celeba_dl = get_celeba_loader(batch_size=batch_size, root='../../datasets/celeba_trunc/')

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

fixed_noise = torch.randn(64, nz, 1, 1, device=device)
real_label = 1.
fake_label = 0.

In [15]:
for e in range(1, epochs + 1):
	gan_hist = fit_dcgan(netG, netD, device, celeba_dl, criterion, e, optimizerG, optimizerD, nz, log_interval=log_interval)

	for stat in gan_hist:
		gan_hist[stat] = np.sum(gan_hist[stat]) / len(gan_hist[stat])
	print(gan_hist)

<class 'torch.Tensor'>
<class 'torch.Tensor'>
<class 'torch.Tensor'>
96
torch.Size([3, 64, 64])


ValueError: expected 4D input (got 3D input)

In [6]:
generate(netG, device, 2, save=True, latent_vecs_batch=fixed_noise)
# pytorch_dcgan_results_1702854757982334094.png

In [29]:
type(imgs), imgs.size()


(torch.Tensor, torch.Size([64, 3, 64, 64]))

In [16]:
imgsnew = np.transpose(torchvision.utils.make_grid(imgs, padding=5, normalize=True).cpu(),(1,2,0))

In [30]:
type(imgsnew), imgsnew.size()

(torch.Tensor, torch.Size([557, 557, 3]))

In [32]:
type(imgsnew.numpy()), imgs.numpy().dtype

(numpy.ndarray, dtype('float32'))

In [16]:
class Bottleneck(nn.Module):

    def __init__(self,in_channels,intermediate_channels,expansion,is_Bottleneck,stride):
        
        """
        Creates a Bottleneck with conv 1x1->3x3->1x1 layers.
        
        Note:
          1. Addition of feature maps occur at just before the final ReLU with the input feature maps
          2. if input size is different from output, select projected mapping or else identity mapping.
          3. if is_Bottleneck=False (3x3->3x3) are used else (1x1->3x3->1x1). Bottleneck is required for resnet-50/101/152
        Args:
            in_channels (int) : input channels to the Bottleneck
            intermediate_channels (int) : number of channels to 3x3 conv 
            expansion (int) : factor by which the input #channels are increased
            stride (int) : stride applied in the 3x3 conv. 2 for first Bottleneck of the block and 1 for remaining

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """

        super(Bottleneck,self).__init__()

        self.expansion = expansion
        self.in_channels = in_channels
        self.intermediate_channels = intermediate_channels
        self.is_Bottleneck = is_Bottleneck
        
        # i.e. if dim(x) == dim(F) => Identity function
        if self.in_channels==self.intermediate_channels*self.expansion:
            self.identity = True
        else:
            self.identity = False
            projection_layer = []
            projection_layer.append(nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=stride, padding=0, bias=False ))
            projection_layer.append(nn.BatchNorm2d(self.intermediate_channels*self.expansion))
            # Only conv->BN and no ReLU
            # projection_layer.append(nn.ReLU())
            self.projection = nn.Sequential(*projection_layer)

        # commonly used relu
        self.relu = nn.ReLU()

        # is_Bottleneck = True for all ResNet 50+
        if self.is_Bottleneck:
            # bottleneck
            # 1x1
            self.conv1_1x1 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 1x1
            self.conv3_1x1 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels*self.expansion, kernel_size=1, stride=1, padding=0, bias=False )
            self.batchnorm3 = nn.BatchNorm2d( self.intermediate_channels*self.expansion )
        
        else:
            # basicblock
            # 3x3
            self.conv1_3x3 = nn.Conv2d(in_channels=self.in_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=stride, padding=1, bias=False )
            self.batchnorm1 = nn.BatchNorm2d(self.intermediate_channels)
            
            # 3x3
            self.conv2_3x3 = nn.Conv2d(in_channels=self.intermediate_channels, out_channels=self.intermediate_channels, kernel_size=3, stride=1, padding=1, bias=False )
            self.batchnorm2 = nn.BatchNorm2d(self.intermediate_channels)

    def forward(self,x):
        # input stored to be added before the final relu
        in_x = x

        if self.is_Bottleneck:
            # conv1x1->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_1x1(x)))
            
            # conv3x3->BN->relu
            x = self.relu(self.batchnorm2(self.conv2_3x3(x)))
            
            # conv1x1->BN
            x = self.batchnorm3(self.conv3_1x1(x))
        
        else:
            # conv3x3->BN->relu
            x = self.relu(self.batchnorm1(self.conv1_3x3(x)))

            # conv3x3->BN
            x = self.batchnorm2(self.conv2_3x3(x))


        # identity or projected mapping
        if self.identity:
            x += in_x
        else:
            x += self.projection(in_x)

        # final relu
        x = self.relu(x)
        
        return x


# Bottleneck(64*4,64,4,stride=1)

def test_Bottleneck():
    x = torch.randn(1,64,112,112)
    model = Bottleneck(64,64,4,True,2)
    print(model(x).shape)
    del model

test_Bottleneck()

torch.Size([1, 256, 56, 56])


In [17]:
class ResNet(nn.Module):

    def __init__(self, resnet_variant,in_channels,num_classes):
        """
        Creates the ResNet architecture based on the provided variant. 18/34/50/101 etc.
        Based on the input parameters, define the channels list, repeatition list along with expansion factor(4) and stride(3/1)
        using _make_blocks method, create a sequence of multiple Bottlenecks
        Average Pool at the end before the FC layer 

        Args:
            resnet_variant (list) : eg. [[64,128,256,512],[3,4,6,3],4,True]
            in_channels (int) : image channels (3)
            num_classes (int) : output #classes 

        Attributes:
            Layer consisting of conv->batchnorm->relu

        """
        super(ResNet,self).__init__()
        self.channels_list = resnet_variant[0]
        self.repeatition_list = resnet_variant[1]
        self.expansion = resnet_variant[2]
        self.is_Bottleneck = resnet_variant[3]

        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=7, stride=2, padding=3, bias=False )
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()

        self.maxpool = nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        # self.block1 = self._make_blocks( 64 , self.channels_list[0], self.repeatition_list[0], self.expansion, self.is_Bottleneck, stride=1 )
        self.block1 = nn.Sequential(*[
            Bottleneck(64,self.channels_list[0],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[0] * self.expansion,self.channels_list[0],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[0] * self.expansion,self.channels_list[0],self.expansion,self.is_Bottleneck,stride=1),
        ])

        # self.block2 = self._make_blocks( self.channels_list[0]*self.expansion , self.channels_list[1], self.repeatition_list[1], self.expansion, self.is_Bottleneck, stride=2 )
        self.block2 = nn.Sequential(*[
            Bottleneck(self.channels_list[0] * self.expansion,self.channels_list[1],self.expansion,self.is_Bottleneck,stride=2),
            Bottleneck(self.channels_list[1] * self.expansion,self.channels_list[1],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[1] * self.expansion,self.channels_list[1],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[1] * self.expansion,self.channels_list[1],self.expansion,self.is_Bottleneck,stride=1),
        ])

        # self.block3 = self._make_blocks( self.channels_list[1]*self.expansion , self.channels_list[2], self.repeatition_list[2], self.expansion, self.is_Bottleneck, stride=2 )
        self.block3 = nn.Sequential(*[
            Bottleneck(self.channels_list[1] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=2),
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[2],self.expansion,self.is_Bottleneck,stride=1),
        ])

        # self.block4 = self._make_blocks( self.channels_list[2]*self.expansion , self.channels_list[3], self.repeatition_list[3], self.expansion, self.is_Bottleneck, stride=2 )
        self.block4 = nn.Sequential(*[
            Bottleneck(self.channels_list[2] * self.expansion,self.channels_list[3],self.expansion,self.is_Bottleneck,stride=2),
            Bottleneck(self.channels_list[3] * self.expansion,self.channels_list[3],self.expansion,self.is_Bottleneck,stride=1),
            Bottleneck(self.channels_list[3] * self.expansion,self.channels_list[3],self.expansion,self.is_Bottleneck,stride=1),
        ])

        self.average_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Linear( self.channels_list[3]*self.expansion , num_classes)



    def forward(self,x):
        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.maxpool(x)
        
        x = self.block1(x)
        
        x = self.block2(x)
        
        x = self.block3(x)
        
        x = self.block4(x)
        
        x = self.average_pool(x)

        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        
        return x

    def _make_blocks(self,in_channels,intermediate_channels,num_repeat, expansion, is_Bottleneck, stride):
        
        """
        Args:
            in_channels : #channels of the Bottleneck input
            intermediate_channels : #channels of the 3x3 in the Bottleneck
            num_repeat : #Bottlenecks in the block
            expansion : factor by which intermediate_channels are multiplied to create the output channels
            is_Bottleneck : status if Bottleneck in required
            stride : stride to be used in the first Bottleneck conv 3x3

        Attributes:
            Sequence of Bottleneck layers

        """
        layers = [] 

        layers.append(Bottleneck(in_channels,intermediate_channels,expansion,is_Bottleneck,stride=stride))
        for num in range(1,num_repeat):
            layers.append(Bottleneck(intermediate_channels*expansion,intermediate_channels,expansion,is_Bottleneck,stride=1))

        return nn.Sequential(*layers)


def test_ResNet(params):
    model = ResNet( params , in_channels=3, num_classes=1000)
    x = torch.randn(1,3,224,224)
    output = model(x)
    print(output.shape)
    return model


model_parameters={}
model_parameters['resnet18'] = ([64,128,256,512],[2,2,2,2],1,False)
model_parameters['resnet34'] = ([64,128,256,512],[3,4,6,3],1,False)
model_parameters['resnet50'] = ([64,128,256,512],[3,4,6,3],4,True)
model_parameters['resnet101'] = ([64,128,256,512],[3,4,23,3],4,True)
model_parameters['resnet152'] = ([64,128,256,512],[3,8,36,3],4,True)

architecture = 'resnet50'
model = test_ResNet(model_parameters[architecture])

torch.Size([1, 1000])


In [5]:
class FCNet(nn.Module):
	
	def __init__(self, layers=[784, 800, 10]):
		super(FCNet, self).__init__()
		self.layers = nn.ModuleList([nn.Linear(a, b) for a, b in zip(layers[:-1], layers[1:])])

	def forward(self, x):
		for layer in self.layers[:-1]:
			x = F.relu(layer(x))
		x = self.layers[-1](x)
		return F.log_softmax(x, dim=1)

train_dl, _, _ = get_mnist_loaders(128)
input_size = next(iter(train_dl))[0].shape[1:]

fcnet = FCNet()
fcnet = fcnet.to(device)
summary(fcnet, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 800]         628,000
            Linear-2                   [-1, 10]           8,010
Total params: 636,010
Trainable params: 636,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 2.43
Estimated Total Size (MB): 2.44
----------------------------------------------------------------


In [10]:
class SCVNet(nn.Module):

	def __init__(self, num_classes=10):
		super().__init__()
		self.conv1 = nn.Sequential(         
			nn.Conv2d(1, 16, 5, 1, 2),
			nn.ReLU(),                                       
			nn.MaxPool2d(2)
		)
		self.conv2 = nn.Sequential(         
			nn.Conv2d(16, 32, 5, 1, 2),
			nn.ReLU(),
			nn.MaxPool2d(2),  
		)
		self.dense = nn.Linear(32 * 7 * 7, 500)
		self.classifier = nn.Linear(500, num_classes)

	def forward(self, x):
		x = self.conv1(x)
		x = self.conv2(x)
		x = torch.flatten(x, 1)
		x = F.relu(self.dense(x))
		return F.log_softmax(self.classifier(x), dim=1)
	
train_dl, _, _ = get_mnist_loaders(128, flatten=False)
input_size = next(iter(train_dl))[0].shape[1:]
print(input_size)

scvnet = SCVNet()
scvnet = scvnet.to(device)
summary(scvnet, input_size)

torch.Size([1, 28, 28])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             416
              ReLU-2           [-1, 16, 28, 28]               0
         MaxPool2d-3           [-1, 16, 14, 14]               0
            Conv2d-4           [-1, 32, 14, 14]          12,832
              ReLU-5           [-1, 32, 14, 14]               0
         MaxPool2d-6             [-1, 32, 7, 7]               0
            Linear-7                  [-1, 500]         784,500
            Linear-8                   [-1, 10]           5,010
Total params: 802,758
Trainable params: 802,758
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.33
Params size (MB): 3.06
Estimated Total Size (MB): 3.39
----------------------------------------------------------------


In [7]:
gen = Generator(3, 100, 64)
gen = gen.to(device)
summary(gen, (100, 1, 1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 4, 4]         819,200
       BatchNorm2d-2            [-1, 512, 4, 4]           1,024
              ReLU-3            [-1, 512, 4, 4]               0
   ConvTranspose2d-4            [-1, 256, 8, 8]       2,097,152
       BatchNorm2d-5            [-1, 256, 8, 8]             512
              ReLU-6            [-1, 256, 8, 8]               0
   ConvTranspose2d-7          [-1, 128, 16, 16]         524,288
       BatchNorm2d-8          [-1, 128, 16, 16]             256
              ReLU-9          [-1, 128, 16, 16]               0
  ConvTranspose2d-10           [-1, 64, 32, 32]         131,072
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
  ConvTranspose2d-13            [-1, 3, 64, 64]           3,072
             Tanh-14            [-1, 3,

In [10]:
disc = Discriminator(3, 64)
disc = disc.to(device)
summary(disc, (3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           3,072
         LeakyReLU-2           [-1, 64, 32, 32]               0
            Conv2d-3          [-1, 128, 16, 16]         131,072
       BatchNorm2d-4          [-1, 128, 16, 16]             256
         LeakyReLU-5          [-1, 128, 16, 16]               0
            Conv2d-6            [-1, 256, 8, 8]         524,288
       BatchNorm2d-7            [-1, 256, 8, 8]             512
         LeakyReLU-8            [-1, 256, 8, 8]               0
            Conv2d-9            [-1, 512, 4, 4]       2,097,152
      BatchNorm2d-10            [-1, 512, 4, 4]           1,024
        LeakyReLU-11            [-1, 512, 4, 4]               0
           Conv2d-12              [-1, 1, 1, 1]           8,192
          Sigmoid-13              [-1, 1, 1, 1]               0
Total params: 2,765,568
Trainable param

In [9]:
fixed_noise = torch.randn(32, 100, 1, 1, device=device)
print(fixed_noise.shape)
gen(fixed_noise).shape

torch.Size([32, 100, 1, 1])


torch.Size([32, 3, 64, 64])

In [19]:
rn = resnet50()
rn.fc = nn.Linear(in_features=2048, out_features=num_classes, bias=True)
rn = rn.to(device)
train_dl, _ = get_cifar10_loaders(128)
input_size = next(iter(train_dl))[0].shape[1:]

summary(rn, input_size)

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13       

In [18]:
rn_native = ResNet(model_parameters['resnet50'], in_channels=3, num_classes=10)
rn_native = rn_native.to(device)
train_dl, _ = get_cifar10_loaders(128)
input_size = next(iter(train_dl))[0].shape[1:]

summary(rn_native, input_size)

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]           4,096
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
           Conv2d-11            [-1, 256, 8, 8]          16,384
      BatchNorm2d-12            [-1, 256, 8, 8]             512
           Conv2d-13       

In [5]:
model = mobilenet_v2()
model.classifier[1] = nn.Linear(in_features=1280, out_features=num_classes, bias=True)
model = model.to(device)

train_dl, _ = get_cifar10_loaders(128)
input_size = next(iter(train_dl))[0].shape[1:]

summary(model, input_size)

Files already downloaded and verified
Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]             864
       BatchNorm2d-2           [-1, 32, 16, 16]              64
             ReLU6-3           [-1, 32, 16, 16]               0
            Conv2d-4           [-1, 32, 16, 16]             288
       BatchNorm2d-5           [-1, 32, 16, 16]              64
             ReLU6-6           [-1, 32, 16, 16]               0
            Conv2d-7           [-1, 16, 16, 16]             512
       BatchNorm2d-8           [-1, 16, 16, 16]              32
  InvertedResidual-9           [-1, 16, 16, 16]               0
           Conv2d-10           [-1, 96, 16, 16]           1,536
      BatchNorm2d-11           [-1, 96, 16, 16]             192
            ReLU6-12           [-1, 96, 16, 16]               0
           Conv2d-13       

In [6]:
celeba = get_celeba_loader(batch_size=96)
next(iter(celeba))[0].shape, len(celeba)

(torch.Size([96, 3, 64, 64]), 2111)

In [3]:
gen = Generator(3, 100, 64)
gen = gen.to(device)
disc = Discriminator(3, 64)
disc = disc.to(device)